In [13]:
# !pip install -q torch_geometric
# !pip install -q class_resolver
# !pip3 install pymatting

In [14]:
# from google.colab import drive
# drive.mount('/content/drive')

In [15]:
!pip install -q torch_geometric
!pip install -q class_resolver
!pip3 install pymatting

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as nnFn
import torch.optim as optim
import numpy as np
import random
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch_geometric.data import Data
from torch_geometric.nn import ChebConv
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, log_loss
)
from torch.utils.data import TensorDataset, DataLoader, Subset
from torch_geometric.data import Data
from torch_geometric.nn import ARMAConv

In [17]:
data = np.load('/home/snu/Downloads/breastmnist_224.npz', allow_pickle=True)
all_images = np.concatenate([data['train_images'], data['val_images'], data['test_images']], axis=0)
all_labels = np.concatenate([data['train_labels'], data['val_labels'], data['test_labels']], axis=0).squeeze()

# Convert to 3-channel RGB
images = all_images.astype(np.float32) / 255.0
images = np.repeat(images[:, None, :, :], 3, axis=1)  # (N, 3, 224, 224)
X_torch = torch.tensor(images)
y_torch = torch.tensor(all_labels).long()
print(f"Raw images: {X_torch.shape}, Labels: {y_torch.shape}")

Raw images: torch.Size([780, 3, 224, 224]), Labels: torch.Size([780])


In [18]:
class0_idx = [i for i in range(len(y_torch)) if y_torch[i] == 0]
class1_idx = [i for i in range(len(y_torch)) if y_torch[i] == 1]

random.seed(42)
sampled_class0 = random.sample(class0_idx, min(1000, len(class0_idx)))
sampled_class1 = random.sample(class1_idx, min(1000, len(class1_idx)))

selected_indices = sampled_class0 + sampled_class1
random.shuffle(selected_indices)

subset_dataset = Subset(TensorDataset(X_torch, y_torch), selected_indices)
subset_loader = DataLoader(subset_dataset, batch_size=64, shuffle=False)

In [19]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
vit = torch.hub.load('facebookresearch/dino:main', 'dino_vitb16')
vit.eval().to(device)

vit_feats, y_list = [], []

with torch.no_grad():
    for imgs, lbls in subset_loader:
        imgs = imgs.to(device)
        feats = vit(imgs)
        vit_feats.append(feats.cpu())
        y_list.extend(lbls.cpu().tolist())

F = torch.cat(vit_feats, dim=0).numpy().astype(np.float32)
y = np.array(y_list).astype(np.int64)

num_nodes, num_feats = F.shape
print(f"Extracted ViT-DINO Features: {F.shape}, Labels: {y.shape}")

Using cache found in /home/snu/.cache/torch/hub/facebookresearch_dino_main


Extracted ViT-DINO Features: (780, 768), Labels: (780,)


In [20]:
import torch.nn as nn
import torch.nn.functional as nnFn
from torch_geometric.nn import GATConv

class GAT_Supervised(nn.Module):
    def __init__(
        self,
        input_dim,
        hidden_dim,
        output_dim,
        heads=2,
        activ="RELU",
        dropout=0.2
    ):
        super(GAT_Supervised, self).__init__()

        # -------- GAT layers --------
        self.gat1 = GATConv(
            input_dim, hidden_dim, heads=heads, concat=True, dropout=dropout
        )
        self.bn1 = nn.BatchNorm1d(hidden_dim * heads)

        self.gat2 = GATConv(
            hidden_dim * heads, hidden_dim, heads=heads, concat=True, dropout=dropout
        )
        self.bn2 = nn.BatchNorm1d(hidden_dim * heads)

        self.gat3 = GATConv(
            hidden_dim * heads, hidden_dim, heads=1, concat=False, dropout=dropout
        )
        self.bn3 = nn.BatchNorm1d(hidden_dim)

        # -------- Classifier --------
        self.fc = nn.Linear(hidden_dim, output_dim)

        self.dropout = nn.Dropout(dropout)

        activations = {
            "SELU": nnFn.selu,
            "SiLU": nnFn.silu,
            "GELU": nnFn.gelu,
            "RELU": nnFn.relu,
            "ELU": nnFn.elu
        }
        self.act = activations.get(activ, nnFn.elu)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.gat1(x, edge_index)
        x = self.bn1(x)
        x = self.act(x)
        x = self.dropout(x)

        x = self.gat2(x, edge_index)
        x = self.bn2(x)
        x = self.act(x)
        x = self.dropout(x)

        x = self.gat3(x, edge_index)
        x = self.bn3(x)
        x = self.act(x)
        x = self.dropout(x)

        logits = self.fc(x)
        return logits


In [21]:
def create_adj(F, alpha=1):
    F_norm = F / np.linalg.norm(F, axis=1, keepdims=True)
    W = np.dot(F_norm, F_norm.T)
    W = np.where(W >= alpha, 1, 0).astype(np.float32)
    W = W / W.max()
    return W

def load_data(adj, node_feats):
    node_feats = torch.from_numpy(node_feats).float()
    edge_index = torch.from_numpy(np.array(np.nonzero((adj > 0))))
    return Data(x=node_feats, edge_index=edge_index)

In [22]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
alpha = 0.73
feats_dim = num_feats
hidden_dim = 256
num_classes = 2
num_epochs = 2000
lr = 0.0001
weight_decay = 1e-4
batch_print_freq = 100

In [23]:
W = create_adj(F, alpha)
data = load_data(W, F).to(device)
print(data)

Data(x=[780, 768], edge_index=[2, 266936])


In [24]:
sss = StratifiedShuffleSplit(n_splits=20, test_size=0.9, random_state=42)

accuracies, precisions, recalls, f1_scores, aucs, ce_losses = [], [], [], [], [], []

for fold, (train_idx, test_idx) in enumerate(sss.split(F, y), start=1):
    print(f"\n=== Fold {fold} ===")

    train_idx_t = torch.from_numpy(train_idx).long().to(device)
    y_train_tensor = torch.from_numpy(y[train_idx]).long().to(device)


    model = GAT_Supervised(feats_dim, hidden_dim, num_classes).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    ce_loss = nn.CrossEntropyLoss()

    for epoch in range(1, num_epochs + 1):
        model.train()
        optimizer.zero_grad()

        logits = model(data)
        loss = ce_loss(logits[train_idx_t], y_train_tensor)

        loss.backward()
        optimizer.step()

        if epoch % batch_print_freq == 0 or epoch == 1:
            model.eval()
            with torch.no_grad():
                preds_train = logits[train_idx_t].argmax(dim=1)
                acc_train = accuracy_score(y_train_tensor.cpu(), preds_train.cpu())
            print(f"Fold {fold} Epoch {epoch}: Loss={loss.item():.6f} | TrainAcc={acc_train:.4f}")

    model.eval()
    with torch.no_grad():
        out = model(data)
        preds = out.argmax(dim=1).cpu().numpy()
        probs = torch.softmax(out, dim=1)[:, 1].cpu().numpy()

    y_test = y[test_idx]
    y_pred_test = preds[test_idx]
    y_pred_test = preds[test_idx]
    y_prob_test = probs[test_idx]

    acc = accuracy_score(y_test, y_pred_test)
    prec = precision_score(y_test, y_pred_test, zero_division=0)
    rec = recall_score(y_test, y_pred_test, zero_division=0)
    f1 = f1_score(y_test, y_pred_test, zero_division=0)
    auc = roc_auc_score(y_test, y_prob_test)
    ce = log_loss(y_test, y_prob_test)

    accuracies.append(acc)
    precisions.append(prec)
    recalls.append(rec)
    f1_scores.append(f1)
    aucs.append(auc)
    ce_losses.append(ce)

    print(f"Fold {fold} → "
          f"Acc={acc:.4f} | Prec={prec:.4f} | Rec={rec:.4f} | "
          f"F1={f1:.4f} | AUC={auc:.4f} | CE Loss={ce:.4f}")


# Final summary
print("\n=== Average Results Across 20 Folds ===")
print(f"Accuracy:  {np.mean(accuracies):.4f} \u00b1 {np.std(accuracies):.4f}")
print(f"Precision: {np.mean(precisions):.4f} \u00b1 {np.std(precisions):.4f}")
print(f"Recall:    {np.mean(recalls):.4f} \u00b1 {np.std(recalls):.4f}")
print(f"F1-score:  {np.mean(f1_scores):.4f} \u00b1 {np.std(f1_scores):.4f}")
print(f"AUC:       {np.mean(aucs):.4f} \u00b1 {np.std(aucs):.4f}")
print(f"CE Loss:   {np.mean(ce_losses):.4f} \u00b1 {np.std(ce_losses):.4f}")


=== Fold 1 ===
Fold 1 Epoch 1: Loss=0.626082 | TrainAcc=0.7692
Fold 1 Epoch 100: Loss=0.167052 | TrainAcc=0.9487
Fold 1 Epoch 200: Loss=0.062563 | TrainAcc=0.9872
Fold 1 Epoch 300: Loss=0.016779 | TrainAcc=1.0000
Fold 1 Epoch 400: Loss=0.042161 | TrainAcc=0.9744
Fold 1 Epoch 500: Loss=0.071083 | TrainAcc=0.9872
Fold 1 Epoch 600: Loss=0.004109 | TrainAcc=1.0000
Fold 1 Epoch 700: Loss=0.010926 | TrainAcc=1.0000
Fold 1 Epoch 800: Loss=0.058513 | TrainAcc=0.9872
Fold 1 Epoch 900: Loss=0.002918 | TrainAcc=1.0000
Fold 1 Epoch 1000: Loss=0.003132 | TrainAcc=1.0000
Fold 1 Epoch 1100: Loss=0.013536 | TrainAcc=0.9872
Fold 1 Epoch 1200: Loss=0.030433 | TrainAcc=0.9872
Fold 1 Epoch 1300: Loss=0.002206 | TrainAcc=1.0000
Fold 1 Epoch 1400: Loss=0.011803 | TrainAcc=0.9872
Fold 1 Epoch 1500: Loss=0.001421 | TrainAcc=1.0000
Fold 1 Epoch 1600: Loss=0.004438 | TrainAcc=1.0000
Fold 1 Epoch 1700: Loss=0.000712 | TrainAcc=1.0000
Fold 1 Epoch 1800: Loss=0.003468 | TrainAcc=1.0000
Fold 1 Epoch 1900: Loss=0.0